### Day 13: Transparent Origami

In [9]:
sample = """
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5""".strip().split('\n')

In [99]:
import numpy as np

def parse_transparent_paper(lines):
    dots, folds = [], []
    for l in lines:
        if l.startswith('fold'):
            axis, pos = l.replace('fold along ', '').split('=')
            folds.append((axis, int(pos)))
        elif l:
            dots.append(tuple(map(int, l.split(','))))
    return dots, folds

class TransparentPaper:
    
    def __init__(self, dots):
        self.size_x = max(map(lambda pos: pos[0], dots)) + 1
        self.size_y = max(map(lambda pos: pos[1], dots)) + 1
        self.pattern = np.zeros((self.size_y, self.size_x), dtype=int)
        for x,y in dots:
            self.pattern[y,x] = 1
    
    def __repr__(self):
        return '\n'.join(''.join('.' if x == 0 else '#' if x == 1 else '' for x in row) for row in self.pattern)
    
    def fold_up(self, pos):
        fold_size = max(pos, self.size_y - pos - 1)
        to_fold = self._pad_up(np.flipud(self.pattern[pos+1:]), fold_size)
        folded = self._pad_up(self.pattern[:pos], fold_size)
        folded[np.where(to_fold == 1)] = 1
        self.pattern = folded
        self.size_x = self.pattern.shape[1]
        self.size_y = self.pattern.shape[0]
        
    def fold_left(self, pos):
        fold_size = max(pos, self.size_x - pos - 1)
        to_fold = self._pad_left(np.fliplr(self.pattern[:,pos+1:]), fold_size)
        folded = self._pad_left(self.pattern[:,:pos], fold_size)
        folded[np.where(to_fold == 1)] = 1
        self.pattern = folded
        self.size_x = self.pattern.shape[1]
        self.size_y = self.pattern.shape[0]
        
    def do_folds(self, folds):
        for fold, pos in folds:
            (self.fold_left if fold == 'x' else self.fold_up)(pos)
            
    def get_visible_dots(self):
        return self.pattern.sum()
    
    @staticmethod
    def _pad_left(arr, target_width):
        pad_width = target_width - arr.shape[1]
        return arr if not pad_width else np.pad(arr, ((0, 0), (pad_width, 0)))
    
    @staticmethod
    def _pad_up(arr, target_width):
        pad_width = target_width - arr.shape[0]
        return arr if not pad_width else np.pad(arr, ((pad_width, 0), (0, 0)))

In [106]:
dots, folds = parse_transparent_paper(sample)
print(f'# dots = {len(dots)} (e.g. {dots[0]})\n# folds = {len(folds)} (e.g. {folds[0]})')

tp = TransparentPaper(dots)
tp.do_folds([folds[0]])
assert tp.get_visible_dots() == 17
tp.do_folds(folds[1:])
print('');display(tp)
tp.fold_left(3)
print('');display(tp)
tp.fold_up(1)
print('');tp

# dots = 18 (e.g. (6, 10))
# folds = 2 (e.g. ('y', 7))



#####
#...#
#...#
#...#
#####
.....
.....

###
#.#
#.#
#.#
###
...
...

...
...
###
#.#
###

In [130]:
# Part 1
dots, folds = parse_transparent_paper(map(str.strip, open('data/input_day13.txt').readlines()))
print(f'# dots = {len(dots)} (e.g. {dots[0]})\n# folds = {len(folds)} (e.g. {folds[0]})')

tp = TransparentPaper(dots)
tp.do_folds([folds[0]])
tp.get_visible_dots()

# dots = 1004 (e.g. (1284, 229))
# folds = 12 (e.g. ('x', 655))


847

In [131]:
# Part 2
tp.do_folds(folds[1:])
print(tp.pattern.shape)
tp  # BCZRCEAB

(6, 40)


###...##..####.###...##..####..##..###..
#..#.#..#....#.#..#.#..#.#....#..#.#..#.
###..#......#..#..#.#....###..#..#.###..
#..#.#.....#...###..#....#....####.#..#.
#..#.#..#.#....#.#..#..#.#....#..#.#..#.
###...##..####.#..#..##..####.#..#.###..